<a href="https://colab.research.google.com/github/TimurMMD/Dissertation_Thesis/blob/main/List_stock_indicators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd
import zipfile
import io
import time

In [2]:
pip install eodhd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 7.8 MB/s eta 0:00:00
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.7.1
    Uninstalling matplotlib-3.7.1:
      Successfully uninstalled matplotlib-3.7.1


In [3]:
symbols = pd.read_csv('sp500_symbols.csv')

In [4]:
symbol_list = symbols['Symbol'].tolist()

In [5]:
symbol_list

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ADBE',
 'AMD',
 'AES',
 'AFL',
 'A',
 'APD',
 'AKAM',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AEE',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AON',
 'APA',
 'AAPL',
 'AMAT',
 'APTV',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'BKR',
 'BALL',
 'BAC',
 'BAX',
 'BDX',
 'BRK.B',
 'BBY',
 'BIIB',
 'BLK',
 'BK',
 'BA',
 'BKNG',
 'BWA',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BF.B',
 'BXP',
 'CHRW',
 'CDNS',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CAT',
 'CBOE',
 'CBRE',
 'CE',
 'COR',
 'CNC',
 'CNP',
 'CF',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA',
 'CAG',
 'COP',
 'ED',
 'STZ',
 'COO',
 'CPRT',
 'GLW',
 'CPAY',
 'COST',
 'CTRA',
 'CCI',
 'CSX',
 'CMI',
 'CVS',
 'DHR',
 'DRI',
 'DVA',
 'DE',
 'DAL',
 'DVN',
 'F

In [6]:
# Function to get data for a specific ticker and data type
def get_financial_data(ticker, data_type, api_token):
    url = f'https://eodhd.com/api/fundamentals/{ticker}.US?filter=Financials::{data_type}::quarterly&api_token=6710b76ee76761.47900881&fmt=json'
    data = requests.get(url).json()
    return pd.DataFrame(data).T



# Function to get earnings history
def get_earnings_data(ticker, api_token):
    url = f'https://eodhd.com/api/fundamentals/{ticker}.US?filter=Earnings::History&api_token=6710b76ee76761.47900881&fmt=json'
    data = requests.get(url).json()
    return pd.DataFrame(data).T



# Function to get stock prices for the ticker
def get_stock_prices(ticker, api_token, start_date='2019-09-24', end_date='2024-07-25'):
    url = f'https://eodhd.com/api/eod/{ticker}.US?from={start_date}&to={end_date}&period=d&api_token=6710b76ee76761.47900881&fmt=json'
    data = requests.get(url).json()
    price_df = pd.DataFrame(data).set_index('date')
    return price_df[['close']]

In [7]:
# Function to process and calculate financial indicators for a given ticker
def process_financial_data(ticker, api_token):

    # Get all relevant financial data
    df_bs = get_financial_data(ticker, "Balance_Sheet", api_token)
    df_is = get_financial_data(ticker, "Income_Statement", api_token)
    df_cf = get_financial_data(ticker, "Cash_Flow", api_token)
    df_earn = get_earnings_data(ticker, api_token)

    # Select relevant columns and set index
    df_bs = df_bs[['date', 'totalCurrentAssets', 'totalCurrentLiabilities', 'inventory', 'totalLiab', 'totalStockholderEquity', 'totalAssets', 'netDebt']].set_index('date')
    df_is = df_is[['date', 'ebit', 'interestExpense', 'totalRevenue', 'costOfRevenue', 'operatingIncome', 'netIncome']].set_index('date')
    df_cf = df_cf[['date', 'freeCashFlow', 'totalCashFromOperatingActivities']].set_index('date')
    df_earn = df_earn[['date', 'epsActual']].set_index('date')
    # Filter this dataset for current dates
    df_earn = df_earn[df_earn.index < '2024-09-30']

    # Merge dataframes
    df = pd.concat([df_bs, df_is, df_cf, df_earn], axis=1)
    df = df.apply(pd.to_numeric, errors='coerce').fillna(0).astype(float)

    # Filter for dates after 2019-09-31
    df = df[df.index > '2019-09-31']

    # Calculate financial ratios
    df['current_ratio'] = df['totalCurrentAssets'] / df['totalCurrentLiabilities']
    df['quick_ratio'] = (df['totalCurrentAssets'] - df['inventory']) / df['totalCurrentLiabilities']
    df['debt_to_equity'] = df['totalLiab'] / df['totalStockholderEquity']
    df['interest_coverage_ratio'] = df['ebit'] / df['interestExpense']
    df['gross_profit_margin'] = (df['totalRevenue'] - df['costOfRevenue']) / df['totalRevenue']
    df['operating_income_margin'] = df['operatingIncome'] / df['totalRevenue']
    df['net_profit_margin'] = df['netIncome'] / df['totalRevenue']
    df['ROA'] = df['netIncome'] / df['totalAssets']
    df['ROE'] = df['netIncome'] / df['totalStockholderEquity']
    df['asset_turnover_ratio'] = df['totalRevenue'] / df['totalAssets']
    df['operating_cash_flow_to_total_debt'] = df['totalCashFromOperatingActivities'] / df['netDebt']

    # Get stock prices
    price_df = get_stock_prices(ticker, api_token)

    # Create a buffer dataframe to fill close price from the previous date on the report date
    df_buffer = pd.concat([df, price_df], axis=1).sort_index().ffill()

    # Merge dataset with close price and other indicators. Sort indexes to calculate the right return
    df = pd.merge(left=df, right=df_buffer['close'], how='left', left_index=True, right_index=True)
    df = df.sort_index()

    # Add PE ratio
    df['PE'] = df['close'] / df['epsActual']

    # Calculate growth rates
    df['revenue_growth'] = df['totalRevenue'].pct_change()
    df['earnings_growth'] = df['netIncome'].pct_change()
    df['return'] = df['close'].pct_change()

    # Keep only necessary columns
    df = df[['freeCashFlow', 'current_ratio', 'quick_ratio', 'debt_to_equity',
       'interest_coverage_ratio', 'gross_profit_margin',
       'operating_income_margin', 'net_profit_margin', 'ROA', 'ROE',
       'asset_turnover_ratio', 'operating_cash_flow_to_total_debt', 'close',
       'PE', 'revenue_growth', 'earnings_growth', 'return']]

    df = df[df.index > '2019-12-31']

    return df


In [8]:
# Function to process multiple tickers and save the result to CSV
# Batch processing with pause (e.g., every 50 tickers)
def process_and_save_tickers_to_zip_in_batches(ticker_list, api_token, zip_filename='financial_data.zip', batch_size=50, delay=60):
    # Create a new in-memory zip archive
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for i in range(0, len(ticker_list), batch_size):
            batch = ticker_list[i:i+batch_size]
            for ticker in batch:
                try:
                    # Process the financial data for each ticker
                    df_final = process_financial_data(ticker, api_token)

                    # Convert the DataFrame to CSV in-memory
                    csv_buffer = io.StringIO()
                    df_final.to_csv(csv_buffer)

                    # Write CSV data to the ZIP archive
                    zipf.writestr(f"{ticker}_final.csv", csv_buffer.getvalue())

                    print(f"Processed and added {ticker} to {zip_filename}")
                except Exception as e:
                    print(f"Error processing {ticker}: {e}")

            # Optional delay after processing each batch to avoid hitting API limits
            print(f"Batch {i//batch_size + 1} completed. Pausing for {delay} seconds.")
            time.sleep(delay)


In [9]:
api_token =  '6710b76ee76761.47900881'

In [10]:
process_and_save_tickers_to_zip_in_batches(symbol_list, api_token)

Processed and added MMM to financial_data.zip
Processed and added AOS to financial_data.zip
Processed and added ABT to financial_data.zip
Processed and added ABBV to financial_data.zip
Processed and added ACN to financial_data.zip
Processed and added ADBE to financial_data.zip
Processed and added AMD to financial_data.zip
Processed and added AES to financial_data.zip
Processed and added AFL to financial_data.zip
Processed and added A to financial_data.zip
Processed and added APD to financial_data.zip
Processed and added AKAM to financial_data.zip
Processed and added ALB to financial_data.zip
Processed and added ARE to financial_data.zip
Processed and added ALGN to financial_data.zip
Processed and added ALLE to financial_data.zip
Processed and added LNT to financial_data.zip
Processed and added ALL to financial_data.zip
Processed and added GOOGL to financial_data.zip
Processed and added GOOG to financial_data.zip
Processed and added MO to financial_data.zip
Processed and added AMZN to f

In [ ]:
def process_and_save_tickers(ticker_list, api_token):
    for ticker in ticker_list:
            df_final = process_financial_data(ticker, api_token)
            # Save to CSV
            df_final.to_csv(f"{ticker}_final.csv")
            print(f"Processed and saved {ticker}")

In [ ]:
ticker_list = ['TSLA']
process_and_save_tickers(ticker_list, api_token)

Processed and saved TSLA
